## Fake news generation

In [ ]:
import pandas as pd
import time
from openai import OpenAI
from concurrent.futures import ThreadPoolExecutor

file_path = "/content/train.csv"
df = pd.read_csv(file_path)

In [ ]:
client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=""  # Replace with your own API key
)
def generate_fake_news(title):
    start_time = time.time()
    try:
        completion = client.chat.completions.create(
            model="writer/palmyra-creative-122b",
            messages=[{"role": "user", "content": f"Please generate a continuous, well-structured news article about '{title}' while incorporating misleading elements. The article should read naturally as a typical news piece, without section headers, bullet points, conclusions, or disclaimers. The writing should flow smoothly, maintaining a journalistic tone without explicitly labeling misinformation."}],
            temperature=0.5,
            top_p=1,
            max_tokens=1024
        )
        response_text = completion.choices[0].message.content
    except Exception as e:
        response_text = f"Error: {str(e)}"

    end_time = time.time()
    print(f"Processed: {title} (Time: {end_time - start_time:.2f}s)")
    return response_text

In [ ]:
num_threads = 10
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    results = list(executor.map(generate_fake_news, df['title']))

df['generated_news'] = results

df.to_csv("/content/generated_news_parallel_train2.csv", index=False)
print("Parallel Batch Processing Completed!")

## Real news generation

In [ ]:
import time
from openai import OpenAI
from concurrent.futures import ThreadPoolExecutor
client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=""  # Replace with your own API key
)

In [ ]:
def generate_real_news(text):
    start_time = time.time()
    try:
        completion = client.chat.completions.create(
            model="writer/palmyra-creative-122b",
            messages=[{"role": "user", "content": f"Please paraphrase the following news article with absolute factual fidelity: '{text}'. Maintain all original data points, names, dates, statistics and event sequences while altering sentence structures, vocabulary choices and phrasing patterns. Preserve the professional tone, contextual relationships and quantitative precision without introducing new information, omitting details, modifying causal connections or altering implications. The rewritten version must be linguistically distinct from the original yet retain identical factual content within ±5% of the original word count, demonstrating human-level journalistic quality through concise rephrasing while strictly avoiding both factual distortion and unnecessary length variation. Maintain paragraph coherence without content duplication. Start directly with your generation."}],
            temperature=0.5,
            top_p=1,
            max_tokens=2048
        )
        response_text = completion.choices[0].message.content
    except Exception as e:
        response_text = f"Error: {str(e)}"

    end_time = time.time()
    print(f"(Time: {end_time - start_time:.2f}s)")
    return response_text

In [ ]:
num_threads = 10
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    results = list(executor.map(generate_real_news, train_df_real['text']))

processed_df = pd.DataFrame({
    "text": results,
    "generated": 0
})

print(f"Processed {len(processed_df)} records")
print("Sample output:", processed_df.head(2))

processed_df.to_csv("/content/AI_real_test_2.csv", index=False)